# General instructions

The goal of the project is to materialize a set of **exploratory workloads** over a real-world, large-scale,  open-domain KG: [WikiData](https://www.wikidata.org/wiki/Wikidata:Main_Page)

An exploratory workload  is composed by a set of queries, where each query is related to the information obtained previously.

An exploratory workload starts with a usually vague, open ended question, and does not assume the person issuing the workload has a clear understanding of the data contained in the target database or its structure.

Remeber that:

1. All the queries must run in the python notebook
2. You can use classes and properties only if you find them via a SPARQL query that must be present in the notebook
3. You do not delete useless queries. Keep everything that is synthatically valid 

```
?p <http://schema.org/name> ?name .
```
    
is the BGP returning a human-readable name of a property or a class in Wikidata.
    
    

In [2]:
## SETUP used later

from SPARQLWrapper import SPARQLWrapper, JSON


prefixString = """
##-c81cee00e2-##
PREFIX wd: <http://www.wikidata.org/entity/> 
PREFIX wdt: <http://www.wikidata.org/prop/direct/> 
PREFIX sc: <http://schema.org/>
"""

# select and construct queries
def run_query(queryString):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-02.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
       results = sparql.query()
       json_results = results.convert()
       if len(json_results['results']['bindings'])==0:
          print("Empty")
          return 0
    
       for bindings in json_results['results']['bindings']:
          print( [ (var, value['value'])  for var, value in bindings.items() ] )

       return len(json_results['results']['bindings'])

    except Exception as e :
        print("The operation failed", e)
    
# ASk queries
def run_ask_query(queryString):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-03.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        return sparql.query().convert()

    except Exception as e :
        print("The operation failed", e)


# Politics Workflow Series ("International Treaties") 

Consider the following exploratory information need:

> You investigating existing international treaties

## Useful URIs for the current workflow


The following are given:

| IRI           | Description   | Role      |
| -----------   | -----------   |-----------|
| `wdt:P1647`   | subproperty   | predicate |
| `wdt:P31`     | instance of   | predicate |
| `wdt:P279`    | subclass      | predicate |
| `wdt:P17`     | country       | predicate |
| `wdt:P27`     | citizenship   | predicate |
| `wdt:P39`     | position held   | predicate |
| `wd:Q131569`  | treaty | node      |
| `wd:Q46`      | Europe        | node |
| `wd:Q38`      | Italy          | node |
| `wd:Q21707860` | Paris Agreement           | node |



Also consider

```
?p wdt:P31/wdt:P279* wd:Q131569  . 
```

is the BGP to retrieve all **treaties**

## Workload Goals


1. Identify the BGP to retrieve treaties that took place in E.U. countries

2. Identify the BGP for obtaining which countries or people signed the treaties

3. How many treaties have been published per year?

4. For how many treaties we know causes or effects?

5. Analyze the number of countries and how many treaties they have signed and their causes
 
   5.1 Are there two countries that signed many common treaties?
   
   5.2 Which are the treaties with most signatories? Which country signed the most treaties?
   
   5.3 Are there countries that did not sign a treaty with the U.S. but signed a treaty with Germany?


1) We have to find predicates that relates a treaty to a location.

In [6]:
queryString = """
SELECT DISTINCT ?p ?pname
WHERE { 
    wd:Q21707860 ?p ?o .
    ?p <http://schema.org/name> ?pname
} 
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P2058'), ('pname', 'depositor')]
[('p', 'http://www.wikidata.org/prop/direct/P3712'), ('pname', 'objective of project or action')]
[('p', 'http://www.wikidata.org/prop/direct/P5799'), ('pname', 'AustLII ID')]
[('p', 'http://www.wikidata.org/prop/direct/P8609'), ('pname', 'China Treaty Database ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1269'), ('pname', 'facet of')]
[('p', 'http://www.wikidata.org/prop/direct/P138'), ('pname', 'named after')]
[('p', 'http://www.wikidata.org/prop/direct/P1542'), ('pname', 'has effect')]
[('p', 'http://www.wikidata.org/prop/direct/P1617'), ('pname', 'BBC Things ID')]
[('p', 'http://www.wikidata.org/prop/direct/P18'), ('pname', 'image')]
[('p', 'http://www.wikidata.org/prop/direct/P227'), ('pname', 'GND ID')]
[('p', 'http://www.wikidata.org/prop/direct/P2671'), ('pname', 'Google Knowledge Graph ID')]
[('p', 'http://www.wikidata.org/prop/direct/P276'), ('pname', 'location')]
[('p', 'http://www.wik

29

The predicate we need is named 'location' (P276).

In [9]:
queryString = """
SELECT DISTINCT ?o ?oname
WHERE { 
    wd:Q21707860 wdt:P31 ?o .
    ?o <http://schema.org/name> ?oname
} 
LIMIT 20
"""

print("Results")
run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q131569'), ('oname', 'treaty')]


1

In [11]:
queryString = """
SELECT DISTINCT ?p ?pname ?location ?locname
WHERE { 
    ?p wdt:P31/wdt:P279* wd:Q131569 .
    ?p wdt:P276 ?location .
    ?p <http://schema.org/name> ?pname .
    ?location <http://schema.org/name> ?locname
} 
LIMIT 20
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/entity/Q2292449'), ('pname', 'Treaty of Craiova'), ('location', 'http://www.wikidata.org/entity/Q168057'), ('locname', 'Craiova')]
[('p', 'http://www.wikidata.org/entity/Q11146'), ('pname', 'Maastricht Treaty'), ('location', 'http://www.wikidata.org/entity/Q1309'), ('locname', 'Maastricht')]
[('p', 'http://www.wikidata.org/entity/Q17986650'), ('pname', 'Convention on the Limitation Period in the International Sale of Goods'), ('location', 'http://www.wikidata.org/entity/Q60'), ('locname', 'New York City')]
[('p', 'http://www.wikidata.org/entity/Q2996265'), ('pname', 'Kinshasa Convention'), ('location', 'http://www.wikidata.org/entity/Q3844'), ('locname', 'Brazzaville')]
[('p', 'http://www.wikidata.org/entity/Q697986'), ('pname', 'Vienna Convention on Road Traffic'), ('location', 'http://www.wikidata.org/entity/Q1741'), ('locname', 'Vienna')]
[('p', 'http://www.wikidata.org/entity/Q28130514'), ('pname', 'Treaty on the Prohibition of Nuclear Weapon

20

We need to find a predicate that relates the location to the area.

In [17]:
queryString = """
SELECT DISTINCT ?country ?countryname
WHERE { 
    wd:Q1741 wdt:P17 ?country .
    ?country <http://schema.org/name> ?countryname
} 
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('country', 'http://www.wikidata.org/entity/Q40'), ('countryname', 'Austria')]


1

In [16]:
queryString = """
SELECT DISTINCT ?p ?pname
WHERE { 
    wd:Q1741 wdt:P17 ?country .
    ?p <http://schema.org/name> ?pname .
    ?country ?p ?o
    FILTER REGEX(?pname, 'continent|has part', 'i')
} 
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P30'), ('pname', 'continent')]


1

In [19]:
queryString = """
SELECT DISTINCT ?p ?pname ?continentname ?continent
WHERE { 
    ?p wdt:P31/wdt:P279* wd:Q131569 .
    ?p wdt:P276 ?location .
    ?p <http://schema.org/name> ?pname .
    ?location <http://schema.org/name> ?locname .
    ?location wdt:P17 ?country .
    ?country wdt:P30 ?continent .
    ?continent <http://schema.org/name> ?continentname
    FILTER REGEX(?continentname, 'Europe', 'i')
    
} 
LIMIT 20
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/entity/Q2292449'), ('pname', 'Treaty of Craiova'), ('continentname', 'Europe'), ('continent', 'http://www.wikidata.org/entity/Q46')]
[('p', 'http://www.wikidata.org/entity/Q11146'), ('pname', 'Maastricht Treaty'), ('continentname', 'Europe'), ('continent', 'http://www.wikidata.org/entity/Q46')]
[('p', 'http://www.wikidata.org/entity/Q697986'), ('pname', 'Vienna Convention on Road Traffic'), ('continentname', 'Europe'), ('continent', 'http://www.wikidata.org/entity/Q46')]
[('p', 'http://www.wikidata.org/entity/Q306535'), ('pname', 'Austrian State Treaty'), ('continentname', 'Europe'), ('continent', 'http://www.wikidata.org/entity/Q46')]
[('p', 'http://www.wikidata.org/entity/Q521923'), ('pname', 'Treaty of Hamburg'), ('continentname', 'Europe'), ('continent', 'http://www.wikidata.org/entity/Q46')]
[('p', 'http://www.wikidata.org/entity/Q617634'), ('pname', 'Anglo-Egyptian treaty of 1936'), ('continentname', 'Europe'), ('continent', 'http://www.wik

20

Here we have all the treaties that took place in Europe

In [22]:
queryString = """
SELECT DISTINCT ?p ?pname ?continentname ?continent
WHERE { 
    ?p wdt:P31/wdt:P279* wd:Q131569 .
    ?p wdt:P276 ?location .
    ?p <http://schema.org/name> ?pname .
    ?location <http://schema.org/name> ?locname .
    ?location wdt:P17 ?country .
    ?country wdt:P30 wd:Q46
    
} 
LIMIT 20
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/entity/Q2292449'), ('pname', 'Treaty of Craiova')]
[('p', 'http://www.wikidata.org/entity/Q11146'), ('pname', 'Maastricht Treaty')]
[('p', 'http://www.wikidata.org/entity/Q697986'), ('pname', 'Vienna Convention on Road Traffic')]
[('p', 'http://www.wikidata.org/entity/Q306535'), ('pname', 'Austrian State Treaty')]
[('p', 'http://www.wikidata.org/entity/Q521923'), ('pname', 'Treaty of Hamburg')]
[('p', 'http://www.wikidata.org/entity/Q617634'), ('pname', 'Anglo-Egyptian treaty of 1936')]
[('p', 'http://www.wikidata.org/entity/Q1923018'), ('pname', 'Treaty of El Pardo')]
[('p', 'http://www.wikidata.org/entity/Q685933'), ('pname', 'Reichstadt Agreement')]
[('p', 'http://www.wikidata.org/entity/Q167457'), ('pname', 'Treaty on the Functioning of the European Union')]
[('p', 'http://www.wikidata.org/entity/Q1160895'), ('pname', 'Treaty of London')]
[('p', 'http://www.wikidata.org/entity/Q154255'), ('pname', 'Munich agreement')]
[('p', 'http://www.wikid

20

Here we count all the treaties in Europe.

In [25]:
queryString = """
SELECT DISTINCT COUNT(?p) as ?howmany ?continentname 
WHERE { 
    ?p wdt:P31/wdt:P279* wd:Q131569 .
    ?p wdt:P276 ?location .
    ?p <http://schema.org/name> ?pname .
    ?location <http://schema.org/name> ?locname .
    ?location wdt:P17 ?country .
    ?country wdt:P30 wd:Q46
    
}
GROUP BY ?continentname
LIMIT 20
"""

print("Results")
run_query(queryString)

Results
[('howmany', '548')]


1

2) We need to retrieve the predicate that we need. In a previus query (the first) we find a propertie 'signatory' (P1891).

In [30]:
queryString = """
SELECT DISTINCT ?pname ?o1name ?o2name
WHERE { 
    ?p wdt:P31/wdt:P279* wd:Q131569  .
    ?p wdt:P6193 ?o1 .
    ?p wdt:P1891 ?o2 .
    ?p <http://schema.org/name> ?pname .
    OPTIONAL{
        ?o1 <http://schema.org/name> ?o1name .
        ?o2 <http://schema.org/name> ?o2name
        }
}
LIMIT 20
"""

print("Results")
run_query(queryString)

Results
[('pname', 'Convention Establishing the World Intellectual Property Organization'), ('o1name', 'Dominican Republic'), ('o2name', 'Gabon')]
[('pname', 'Convention Establishing the World Intellectual Property Organization'), ('o1name', 'Dominican Republic'), ('o2name', 'Ivory Coast')]
[('pname', 'Convention Establishing the World Intellectual Property Organization'), ('o1name', 'Dominican Republic'), ('o2name', 'Cameroon')]
[('pname', 'Convention Establishing the World Intellectual Property Organization'), ('o1name', 'Dominican Republic'), ('o2name', 'Madagascar')]
[('pname', 'Convention Establishing the World Intellectual Property Organization'), ('o1name', 'Dominican Republic'), ('o2name', 'Morocco')]
[('pname', 'Convention Establishing the World Intellectual Property Organization'), ('o1name', 'Dominican Republic'), ('o2name', 'Niger')]
[('pname', 'Convention Establishing the World Intellectual Property Organization'), ('o1name', 'Dominican Republic'), ('o2name', 'Senegal')]
[

20

3) We see in a previus query (the first) the predicate 'publication date'.

In [31]:
queryString = """
SELECT DISTINCT ?pname ?date
WHERE { 
    ?p wdt:P31/wdt:P279* wd:Q131569  .
    ?p wdt:P577 ?date .
    ?p <http://schema.org/name> ?pname
}
LIMIT 20
"""

print("Results")
run_query(queryString)

Results
[('pname', 'International Covenant on Economic, Social and Cultural Rights'), ('date', '1966-12-16T00:00:00Z')]
[('pname', 'Convention on the Rights of the Child'), ('date', '1989-11-20T00:00:00Z')]
[('pname', 'Treaty on the Prohibition of Nuclear Weapons'), ('date', '2017-07-07T00:00:00Z')]
[('pname', 'Secret Additional Protocol to the Molotov–Ribbentrop Pact'), ('date', '1939-08-23T00:00:00Z')]
[('pname', 'Second Geneva Convention'), ('date', '1949-08-12T00:00:00Z')]
[('pname', 'Agreement between the Parties to the North Atlantic Treaty regarding the Status of their Forces'), ('date', '1951-06-19T00:00:00Z')]
[('pname', 'Minamata Convention on Mercury'), ('date', '2013-10-10T00:00:00Z')]
[('pname', 'European Agreement concerning the International Carriage of Dangerous Goods by Road'), ('date', '1957-09-30T00:00:00Z')]
[('pname', 'Munich agreement'), ('date', '1939-09-30T00:00:00Z')]
[('pname', 'United Nations Framework Convention on Climate Change'), ('date', '1992-05-09T00:0

20

In [36]:
queryString = """
SELECT DISTINCT COUNT(?pname) as ?howmanytreaties ?year
WHERE { 
    ?p wdt:P31/wdt:P279* wd:Q131569  .
    ?p wdt:P577 ?date .
    ?p <http://schema.org/name> ?pname
}
GROUP BY (year(?date) as ?year)
ORDER BY DESC (?year)
LIMIT 30
"""

print("Results")
run_query(queryString)

Results
[('howmanytreaties', '3'), ('year', '2020')]
[('howmanytreaties', '1'), ('year', '2017')]
[('howmanytreaties', '1'), ('year', '2015')]
[('howmanytreaties', '1'), ('year', '2014')]
[('howmanytreaties', '3'), ('year', '2013')]
[('howmanytreaties', '1'), ('year', '2010')]
[('howmanytreaties', '1'), ('year', '2005')]
[('howmanytreaties', '1'), ('year', '2004')]
[('howmanytreaties', '1'), ('year', '2003')]
[('howmanytreaties', '1'), ('year', '2001')]
[('howmanytreaties', '3'), ('year', '2000')]
[('howmanytreaties', '3'), ('year', '1999')]
[('howmanytreaties', '2'), ('year', '1998')]
[('howmanytreaties', '1'), ('year', '1997')]
[('howmanytreaties', '2'), ('year', '1996')]
[('howmanytreaties', '1'), ('year', '1992')]
[('howmanytreaties', '1'), ('year', '1991')]
[('howmanytreaties', '1'), ('year', '1990')]
[('howmanytreaties', '2'), ('year', '1989')]
[('howmanytreaties', '1'), ('year', '1988')]
[('howmanytreaties', '1'), ('year', '1987')]
[('howmanytreaties', '1'), ('year', '1985')]
[(

30

4) Like always in the first query we can find the predicates we need, in this case are the ones called 'has cause' (P828) and 'has effect' (P1542).

In [40]:
queryString = """
SELECT DISTINCT ?causename ?effectname ?pname
WHERE { 
    ?p wdt:P31/wdt:P279* wd:Q131569  .
    ?p <http://schema.org/name> ?pname .
    OPTIONAL{
    ?p wdt:P828 ?cause .
    ?cause <http://schema.org/name> ?causename .
    }
    OPTIONAL{
    ?p wdt:P1542 ?effect .
    ?effect <http://schema.org/name> ?effectname .
    }
}

LIMIT 30
"""

print("Results")
run_query(queryString)

Results
[('causename', 'Constitution of India'), ('effectname', 'Republics in the Commonwealth of Nations'), ('pname', 'London Declaration')]
[('causename', 'Constitution of India'), ('effectname', 'Head of the Commonwealth'), ('pname', 'London Declaration')]
[('causename', 'climate change'), ('effectname', 'United States withdrawal from the Paris Agreement'), ('pname', 'Paris Agreement')]
[('causename', 'Sudeten Crisis'), ('pname', 'Munich agreement')]
[('causename', 'Fourth Crusade'), ('pname', 'Partitio terrarum imperii Romaniae')]
[('causename', 'French and Indian War'), ('pname', 'Treaty of Fontainebleau')]
[('causename', '6th BRICS summit'), ('pname', 'BRICS Contingent Reserve Arrangement')]
[('causename', 'War of the Second Coalition'), ('pname', 'Treaty of Paris')]
[('causename', "L'Escalade"), ('pname', 'Traité de Saint-Julien')]
[('causename', '2004 South American Summit'), ('pname', 'Cusco Declaration')]
[('causename', 'Hague Convention'), ('pname', 'Hague Conventions of 189

30

Here we count the treaties with a cause or a effect.

In [62]:
queryString = """
SELECT DISTINCT COUNT(?p) as ?howmany
WHERE { 
    ?p wdt:P31/wdt:P279* wd:Q131569  .
    FILTER EXISTS{?p wdt:P828/wdt:P1542* ?o1}
}

"""
print("Results")
run_query(queryString)

Results
[('howmany', '41')]


1

5) Let's see how many treaties the countries have signed.

In [65]:
queryString = """
SELECT DISTINCT COUNT(?p) as ?howmanytreaties ?countryname
WHERE { 
    ?p wdt:P31/wdt:P279* wd:Q131569 .
    ?p wdt:P1891 ?country .
    ?country <http://schema.org/name> ?countryname
}
GROUP BY ?countryname

LIMIT 30
"""
print("Results")
run_query(queryString)

Results
[('howmanytreaties', '13'), ('countryname', 'South Korea')]
[('howmanytreaties', '1'), ('countryname', 'Charles, Duke of Orléans')]
[('howmanytreaties', '2'), ('countryname', 'Peter II, Grand Duke of Oldenburg')]
[('howmanytreaties', '1'), ('countryname', 'Elizabeth I of England')]
[('howmanytreaties', '3'), ('countryname', 'Joachim von Ribbentrop')]
[('howmanytreaties', '2'), ('countryname', 'Electorate of Bavaria')]
[('howmanytreaties', '3'), ('countryname', 'Mozambique')]
[('howmanytreaties', '3'), ('countryname', 'Uganda')]
[('howmanytreaties', '1'), ('countryname', 'Saxony')]
[('howmanytreaties', '23'), ('countryname', 'Slovenia')]
[('howmanytreaties', '3'), ('countryname', 'Empire of Brazil')]
[('howmanytreaties', '3'), ('countryname', 'Russian Socialist Federative Soviet Republic')]
[('howmanytreaties', '2'), ('countryname', 'Uzbekistan')]
[('howmanytreaties', '14'), ('countryname', 'Liechtenstein')]
[('howmanytreaties', '1'), ('countryname', 'Tuvalu')]
[('howmanytreatie

30

In [74]:
queryString = """
SELECT DISTINCT COUNT(?p) as ?howmany ?country1name ?country2name
WHERE { 
    ?p wdt:P31/wdt:P279* wd:Q131569 .
    ?p wdt:P1891 ?country1 .
    ?p wdt:P1891 ?country2 .
    ?country1 <http://schema.org/name> ?country1name .
    ?country2 <http://schema.org/name> ?country2name .
    FILTER(?country1name != ?country2name)
    
}
GROUP BY ?country1name ?country2name
ORDER BY DESC (COUNT(?p))
LIMIT 30
"""
print("Results")
run_query(queryString)

Results
[('howmany', '37'), ('country1name', 'France'), ('country2name', 'Germany')]
[('howmany', '37'), ('country1name', 'Germany'), ('country2name', 'France')]
[('howmany', '36'), ('country1name', 'France'), ('country2name', 'Italy')]
[('howmany', '36'), ('country1name', 'Italy'), ('country2name', 'France')]
[('howmany', '33'), ('country1name', 'Belgium'), ('country2name', 'France')]
[('howmany', '33'), ('country1name', 'France'), ('country2name', 'Belgium')]
[('howmany', '32'), ('country1name', 'France'), ('country2name', 'Switzerland')]
[('howmany', '32'), ('country1name', 'Luxembourg'), ('country2name', 'France')]
[('howmany', '32'), ('country1name', 'France'), ('country2name', 'Luxembourg')]
[('howmany', '32'), ('country1name', 'Spain'), ('country2name', 'France')]
[('howmany', '32'), ('country1name', 'Switzerland'), ('country2name', 'France')]
[('howmany', '32'), ('country1name', 'France'), ('country2name', 'Spain')]
[('howmany', '32'), ('country1name', 'United Kingdom'), ('coun

30

5.2) Treaties with highest number of signatories.

In [75]:
queryString = """
SELECT DISTINCT COUNT(?country) as ?howmany ?pname
WHERE { 
    ?p wdt:P31/wdt:P279* wd:Q131569 .
    ?p wdt:P1891 ?country .
    ?p <http://schema.org/name> ?pname
    
}
GROUP BY ?pname
ORDER BY DESC (COUNT(?country))
LIMIT 3
"""
print("Results")
run_query(queryString)

Results
[('howmany', '97'), ('pname', 'Universal Copyright Convention')]
[('howmany', '86'), ('pname', 'Treaty on the Prohibition of Nuclear Weapons')]
[('howmany', '80'), ('pname', 'Marrakesh Treaty to Facilitate Access to Published Works for Persons Who Are Blind, Visually Impaired or Otherwise Print Disabled')]


3

In [ ]:
Countries wich have signed the highest number of treaties.

In [76]:
queryString = """
SELECT DISTINCT COUNT(?p) as ?howmany ?countryname
WHERE { 
    ?p wdt:P31/wdt:P279* wd:Q131569 .
    ?p wdt:P1891 ?country .
    ?country <http://schema.org/name> ?countryname
    
}
GROUP BY ?countryname
ORDER BY DESC (COUNT(?p))
LIMIT 3
"""
print("Results")
run_query(queryString)

Results
[('howmany', '84'), ('countryname', 'France')]
[('howmany', '79'), ('countryname', 'Spain')]
[('howmany', '73'), ('countryname', 'United States of America')]


3

5.3) We have to find the node of United States and Germany.

In [84]:
queryString = """
SELECT DISTINCT ?o ?oname
WHERE { 
    ?p wdt:P17 ?o .
    ?o <http://schema.org/name> ?oname
    FILTER REGEX(?oname, '^United States of America$', 'i')
    
}

"""
print("Results")
run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q30'), ('oname', 'United States of America')]
[('o', 'http://www.wikidata.org/entity/Q19971019'), ('oname', 'United States of America')]
[('o', 'http://www.wikidata.org/entity/Q99281400'), ('oname', 'United States of America')]


3

I choose the one with code P30.

In [83]:
queryString = """
SELECT DISTINCT ?o ?oname
WHERE { 
    ?p wdt:P17 ?o .
    ?o <http://schema.org/name> ?oname
    FILTER REGEX(?oname, '^Germany$', 'i')
    
}

"""
print("Results")
run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q183'), ('oname', 'Germany')]


1

Here we find the treaties where there isn't USA and where there is Germany and return all the country that signed these treaties.

In [95]:
queryString = """
SELECT DISTINCT ?countryname
WHERE { 
    ?p wdt:P31/wdt:P279* wd:Q131569 .
    FILTER NOT EXISTS {?p wdt:P1891 wd:Q30}
    FILTER EXISTS {?p wdt:P1891 wd:Q183}
    ?p <http://schema.org/name> ?pname .
    ?p wdt:P1891 ?country .
    ?country <http://schema.org/name> ?countryname
    
}

LIMIT 100
"""
print("Results")
run_query(queryString)

Results
[('countryname', 'Danish Realm')]
[('countryname', 'Guinea')]
[('countryname', 'Morocco')]
[('countryname', 'Niger')]
[('countryname', 'Senegal')]
[('countryname', 'Seychelles')]
[('countryname', 'Kenya')]
[('countryname', 'France')]
[('countryname', 'United Kingdom')]
[('countryname', "People's Republic of China")]
[('countryname', 'Brazil')]
[('countryname', 'Russia')]
[('countryname', 'Holy See')]
[('countryname', 'Canada')]
[('countryname', 'European Coal and Steel Community')]
[('countryname', 'Japan')]


16